#### **Fundamentos de Bancos de Dados Relacionais e NoSQL**

#### **Conteúdo - Bases e Notebook da aula**

https://github.com/FIAP/Pos_Tech_DTAT/tree/main/Fase%203

#### **Importação de pacotes, bibliotecas e funções (def)**

In [14]:
#!pip uninstall boto3 -y
!pip uninstall botocore -y
# !pip install boto3==1.40.26 
# !pip install botocore==1.40.26
# # !pip install s3fs



Found existing installation: botocore 1.40.26
Uninstalling botocore-1.40.26:
  Successfully uninstalled botocore-1.40.26


In [15]:
#!pip install boto3==1.40.26 
!pip install botocore==1.40.26

  Using cached botocore-1.40.26-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.40.26-py3-none-any.whl (14.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.40.26 which is incompatible.


In [16]:
import boto3, botocore
print("boto3:", boto3.__version__)
print("botocore:", botocore.__version__)


boto3: 1.40.26
botocore: 1.40.26


In [17]:
# Importar biblioteca completa
import boto3
import pandas as pd
import os
import plotly.express as px
import requests
import botocore
import psycopg2
import numpy as np
import duckdb
import mongomock
import fakeredis
import uuid
import json

# Importar função especifica de um módulo
from botocore.exceptions import BotoCoreError, ClientError
from sqlalchemy import create_engine, text, inspect
from dotenv import load_dotenv
from io import StringIO
from datetime import datetime
from pprint import pprint
from astrapy import DataAPIClient
# from cassandra.cluster import Cluster
# from cassandra.auth import PlainTextAuthProvider

#### **Testar conexão AWS via Python**

In [18]:
# Validar conexão
try:
    session = boto3.Session(profile_name="default")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAZI2LFHB2TERRDYNA4:user4377774=maradinidiego16@gmail.com
Account: 637423401077
Arn: arn:aws:sts::637423401077:assumed-role/voclabs/user4377774=maradinidiego16@gmail.com


##### **Conectar ao PostgreSQL via RDS + Executar Comandos SQL**

In [19]:

load_dotenv("Users\dmaradini\.aws\credentials")

aws_access_key_id       = os.getenv("aws_access_key_id")
aws_secret_access_key   = os.getenv("aws_secret_access_key")
aws_session_token       = os.getenv("aws_session_token")  

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dmaradini\AppData\Local\Temp\ipykernel_2944\2708774549.py:1: SyntaxWarning: invalid escape sequence '\d'
  load_dotenv("Users\dmaradini\.aws\credentials")


In [20]:
import boto3
from botocore.exceptions import ClientError
import json 

session = boto3.Session(
    aws_access_key_id       = aws_access_key_id,
    aws_secret_access_key  = aws_secret_access_key,
    aws_session_token      = aws_session_token
)

bucket_name = "techchallange-637423401077"
s3_prefix = "raw"

s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1"

# Criar bucket se não existir
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ("404", "NoSuchBucket"):
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        # Cria bucket público já na criação
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name, ACL='public-read')
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region},
                ACL='public-read'
            )
        print(f"Bucket '{bucket_name}' criado com sucesso e é público.\n")
    else:
        print(f"Erro ao acessar o bucket: {e}")
        raise

# -----------------------------
# Garantir que ACLs públicas não sejam bloqueadas
# -----------------------------
s3.put_public_access_block(
    Bucket=bucket_name,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': False,
        'IgnorePublicAcls': False,
        'BlockPublicPolicy': False,  # mantém políticas públicas bloqueadas (seguro)
        'RestrictPublicBuckets': False
    }
)


# Criar política de bucket pública
public_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(public_policy))
print(f"Bucket '{bucket_name}' configurado como público.")


Bucket 'techchallange-637423401077' já existe e está acessível.
Bucket 'techchallange-637423401077' configurado como público.


In [21]:
import pandas as pd
import boto3
import numpy as npfrp
from datetime import datetime
from io import StringIO
from botocore.exceptions import ClientError


session = boto3.Session(
    aws_access_key_id       =   aws_access_key_id
    ,aws_secret_access_key  =   aws_secret_access_key
    ,aws_session_token      =   aws_session_token
)



bucket_name = "techchallange-637423401077"
s3_prefix = "raw"



print("Conexão estabelecida.")

s3 = session.client('s3')
region = s3.meta.region_name or "us-east-1"

try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' já existe e está acessível.")
except ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code in ("404", "NoSuchBucket"):
        print(f"Bucket '{bucket_name}' não existe, criando...\n")
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket_name)
            
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' criado com sucesso.\n")
    else:
        print(f"Erro ao acessar o bucket: {e}")
        raise



# Criar política de bucket pública
public_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{bucket_name}/*"
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(public_policy))
print(f"Bucket '{bucket_name}' configurado como público.")

Conexão estabelecida.
Bucket 'techchallange-637423401077' já existe e está acessível.
Bucket 'techchallange-637423401077' configurado como público.


### CAMADA BRONZE (EXTRACT)

In [22]:
import pandas as pd
import boto3
from io import BytesIO, StringIO
from zipfile import ZipFile
import requests

bucket_name = "techchallange-637423401077"
s3_prefix = "raw"

s3 = boto3.client("s3")

# URLs dos arquivos do IBGE
urls = {
    "PNAD_COVID_052020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_052020.zip",
    "PNAD_COVID_062020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_062020.zip",
    "PNAD_COVID_072020": "https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_PNAD_COVID19/Microdados/Dados/PNAD_COVID_072020.zip",
}

for tabela, url in urls.items():
    print(f"Processando {tabela}...")

    r = requests.get(url)
    r.raise_for_status()

    if "Dicionario" in tabela:
        # Salvar o XLS original no S3
        s3_key = f"{s3_prefix}/{tabela}.xls"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=r.content  # conteúdo binário do XLS
        )
        print(f"Dicionário salvo no S3 em s3://{bucket_name}/{s3_key}")

    else:
        # Abrir o ZIP e extrair o CSV
        with ZipFile(BytesIO(r.content)) as z:
            csv_name = z.namelist()[0]
            with z.open(csv_name) as f:
                df = pd.read_csv(f, sep=";", encoding="latin1")

        # Converter para CSV em memória
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)

        s3_key = f"{s3_prefix}/{tabela}.csv"
        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=csv_buffer.getvalue()
        )
        print(f"Dados salvos no S3 em s3://{bucket_name}/{s3_key}")

print("Exportação concluída com sucesso.")


Processando PNAD_COVID_052020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_052020.csv
Processando PNAD_COVID_062020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_062020.csv
Processando PNAD_COVID_072020...
Dados salvos no S3 em s3://techchallange-637423401077/raw/PNAD_COVID_072020.csv
Exportação concluída com sucesso.


In [23]:
import boto3

session = boto3.Session(profile_name="default")
s3 = session.client("s3")
print("Consegui conectar no S3!")


Consegui conectar no S3!


### CAMADA SILVER (TRANFORM)

In [24]:
import pandas as pd
import requests
from io import StringIO

# Lista de arquivos CSV no bucket público
bronze_files = [
    "PNAD_COVID_052020.csv",
    "PNAD_COVID_062020.csv",
    "PNAD_COVID_072020.csv"
]

# URL base do bucket público
base_url = "https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/"

lista_de_dataframes = []

for file_name in bronze_files:
    file_url = f"{base_url}{file_name}"
    print(f"Lendo {file_url}")
    
    # Faz download do CSV como texto
    response = requests.get(file_url)
    response.raise_for_status()  # garante que deu certo
    
    # Lê o CSV como texto cru
    df_raw = pd.read_csv(StringIO(response.text), header=None)
    
    # Divide a única coluna pelo separador vírgula
    df_temp = df_raw[0].str.split(",", expand=True)
    
    # Define a primeira linha como header
    df_temp.columns = df_temp.iloc[0]
    df_temp = df_temp.drop(0).reset_index(drop=True)
    
    lista_de_dataframes.append(df_temp)

# Consolidar todos os dataframes
df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)


Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_052020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_062020.csv
Lendo https://techchallange-637423401077.s3.us-east-1.amazonaws.com/raw/PNAD_COVID_072020.csv


In [25]:
import boto3
import pandas as pd
from io import BytesIO

# Configurações do bucket
bucket_name = "techchallange-637423401077"
s3_key = "silver/df_consolidado.parquet"

# Conexão S3
s3_client = boto3.client("s3")  # precisa das credenciais configuradas no seu ambiente

# Salva o DataFrame em um buffer Parquet
buffer = BytesIO()
df_consolidado.to_parquet(buffer, index=False)

# Envia para o S3
buffer.seek(0)
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=buffer.getvalue())

print(f"DataFrame salvo em s3://{bucket_name}/{s3_key}")


DataFrame salvo em s3://techchallange-637423401077/silver/df_consolidado.parquet


In [26]:
df_consolidado_parquet_aws = pd.read_parquet("https://techchallange-637423401077.s3.us-east-1.amazonaws.com/silver/df_consolidado.parquet")
#pd.read_parquet('Dados.parquet') #

In [27]:
df_consolidado_parquet = df_consolidado_parquet_aws

In [28]:
import pandas as pd

# Dicionário de renomeação completo
mapa_colunas = {
    # Colunas fixas
    "ANO": "ano",
    "V1013": "mes_pesquisa",
    "V1012": "semana_mes",
    "UF": "uf",
    "CAPITAL": "capital",

    # Colunas desejadas
    "A002": "idade",
    "A003": "sexo",
    "A004": "raca_cor",
    "A006B": "aulas_presenciais",
    "B008": "fez_exame",
    "B009A": "exame_swab",
    "B009C": "exame_furo_dedo",
    "B009E": "exame_veia_braco",
    "A005": "escolaridade",
    "A006": "frequenta_escola",
    "A006A": "tipo_escola",
    "B0011": "teve_febre_semana_passada",
    "B0012": "teve_tosse_semana_passada",
    "B0013": "teve_dor_garganta_semana_passada",
    "B0014": "teve_dificuldade_respirar",
    "B0015": "teve_dor_cabeca",
    "B0016": "teve_dor_peito",
    "B0017": "teve_nausea",
    "B0018": "teve_nariz_entupido",
    "B0019": "teve_fadiga",
    "B00110": "teve_dor_olhos",
    "B00111": "teve_perda_cheiro_sabor",
    "B00112": "teve_dor_muscular",
    "B00113": "teve_diarreia"
}


colunas_existentes = [col for col in mapa_colunas.keys() if col in df_consolidado_parquet.columns]

# Renomeia apenas essas colunas
df_consolidado_parquet.rename(columns={col: mapa_colunas[col] for col in colunas_existentes}, inplace=True)


df_consolidado_parquet = df_consolidado_parquet[[mapa_colunas[col] for col in colunas_existentes]]



In [29]:
mapa_sim_nao = {
    1: "Sim",
    2: "Não",
    3: "Não sabe",
    9: "Ignorado"
}

mapa_uf = {
    11: "Rondônia", 12: "Acre", 13: "Amazonas", 14: "Roraima", 15: "Pará", 16: "Amapá", 17: "Tocantins",
    21: "Maranhão", 22: "Piauí", 23: "Ceará", 24: "Rio Grande do Norte", 25: "Paraíba", 26: "Pernambuco",
    27: "Alagoas", 28: "Sergipe", 29: "Bahia", 31: "Minas Gerais", 32: "Espírito Santo", 33: "Rio de Janeiro",
    35: "São Paulo", 41: "Paraná", 42: "Santa Catarina", 43: "Rio Grande do Sul", 50: "Mato Grosso do Sul",
    51: "Mato Grosso", 52: "Goiás", 53: "Distrito Federal"
}


df_consolidado_parquet = df_consolidado_parquet.copy()

df_consolidado_parquet["uf"] = df_consolidado_parquet["uf"].astype(int)
df_consolidado_parquet["uf"] = df_consolidado_parquet["uf"].replace(mapa_uf)




# # Substituir colunas de Sim/Não (exemplo: febre, tosse, exames, etc)
colunas_sim_nao = [
    "fez_exame", "exame_swab", "exame_furo_dedo", "exame_veia_braco",
    "teve_febre_semana_passada", "teve_tosse_semana_passada", "teve_dor_garganta_semana_passada",
    "teve_dificuldade_respirar", "teve_dor_cabeca", "teve_dor_peito", "teve_nausea",
    "teve_nariz_entupido", "teve_fadiga", "teve_dor_olhos", "teve_perda_cheiro_sabor",
    "teve_dor_muscular", "teve_diarreia", "aulas_presenciais", "frequenta_escola"
]
colunas_sim_nao = [
    "fez_exame", "exame_swab", "exame_furo_dedo", "exame_veia_braco",
    "teve_febre_semana_passada", "teve_tosse_semana_passada", "teve_dor_garganta_semana_passada",
    "teve_dificuldade_respirar", "teve_dor_cabeca", "teve_dor_peito", "teve_nausea",
    "teve_nariz_entupido", "teve_fadiga", "teve_dor_olhos", "teve_perda_cheiro_sabor",
    "teve_dor_muscular", "teve_diarreia", "aulas_presenciais", "frequenta_escola"
]

# Criar um dicionário de substituição com strings
mapa_sim_nao_str = {str(k): v for k, v in mapa_sim_nao.items()}

for col in colunas_sim_nao:
    if col in df_consolidado_parquet.columns:
        # Substituir valores apenas se não forem nulos
        df_consolidado_parquet[col] = df_consolidado_parquet[col].apply(
            lambda x: mapa_sim_nao_str.get(str(x), x) if pd.notna(x) else x
        )

# Dicionário de substituição
mapa_sexo = {
    "1": "Homem",
    "2": "Mulher"
}

# Substituir valores na coluna 'sexo' (somente se a coluna existir)
if "sexo" in df_consolidado_parquet.columns:
    df_consolidado_parquet["sexo"] = df_consolidado_parquet["sexo"].astype(str).replace(mapa_sexo)


# Dicionário de mapeamento para raca_cor
mapa_raca_cor = {
    "1": "Branca",
    "2": "Preta",
    "3": "Amarela",
    "4": "Parda",
    "5": "Indígena",
    "9": "Ignorado"
}

# Aplicar mapeamento no dataframe
if "raca_cor" in df_consolidado_parquet.columns:
    df_consolidado_parquet["raca_cor"] = df_consolidado_parquet["raca_cor"].astype(str).replace(mapa_raca_cor)



In [30]:
df_consolidado_parquet["sexo"] = df_consolidado_parquet["sexo"].astype("category")
df_consolidado_parquet["mes_pesquisa"] = df_consolidado_parquet["mes_pesquisa"].astype("category")
df_consolidado_parquet["raca_cor"] = df_consolidado_parquet["raca_cor"].astype("category")


In [31]:
import boto3
import pandas as pd
from io import BytesIO

# Seu DataFrame
# df_consolidado

# Configurações do bucket
bucket_name = "techchallange-637423401077"
s3_key = "silver/df_consolidado_Tratado.parquet"

# Conexão S3
s3_client = boto3.client("s3")  # precisa das credenciais configuradas no seu ambiente

# Salva o DataFrame em um buffer Parquet
buffer = BytesIO()
df_consolidado_parquet.to_parquet(buffer, index=False)

# Envia para o S3
buffer.seek(0)
s3_client.put_object(Bucket=bucket_name, Key=s3_key, Body=buffer.getvalue())

print(f"DataFrame salvo em s3://{bucket_name}/{s3_key}")


DataFrame salvo em s3://techchallange-637423401077/silver/df_consolidado_Tratado.parquet


### CAMADA OURO (LOAD)


In [32]:
# Converter sintomas "Sim"/"Não" em 1/0
sintomas = [ 'teve_febre_semana_passada', 'teve_tosse_semana_passada',
       'teve_dor_garganta_semana_passada', 'teve_dificuldade_respirar',
       'teve_dor_cabeca', 'teve_dor_peito', 'teve_nausea',
       'teve_nariz_entupido', 'teve_fadiga', 'teve_dor_olhos',
       'teve_perda_cheiro_sabor', 'teve_dor_muscular', 'teve_diarreia']


In [33]:
# df = df_consolidado_parquet.copy()

# df = df.groupby("raca_cor")[sintomas].sum().reset_index()

In [34]:
df_sexo = df_consolidado_parquet.copy()

for col in sintomas:
    df_sexo[col] = df_sexo[col].map({"Sim": 1, "Não": 0})

# Agrupar por sexo
df_sexo_agrupado = df_sexo.groupby("sexo")[sintomas].sum().reset_index()


C:\Users\dmaradini\AppData\Local\Temp\ipykernel_2944\2682411368.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sexo_agrupado = df_sexo.groupby("sexo")[sintomas].sum().reset_index()


In [35]:
df_mes = df_consolidado_parquet.copy()
for col in sintomas:
    df_mes[col] = df_mes[col].map({"Sim": 1, "Não": 0})

df_mes_agrupado = df_mes.groupby("mes_pesquisa")[sintomas].sum().reset_index()


C:\Users\dmaradini\AppData\Local\Temp\ipykernel_2944\2795615387.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_mes_agrupado = df_mes.groupby("mes_pesquisa")[sintomas].sum().reset_index()


In [36]:
df_sexo_mes = df_consolidado_parquet.copy()
for col in sintomas:
    df_sexo_mes[col] = df_sexo_mes[col].map({"Sim": 1, "Não": 0})

# Agrupar por sexo e mês
df_sexo_mes_agrupado = df_sexo_mes.groupby(["mes_pesquisa", "sexo"])[sintomas].sum().reset_index()


C:\Users\dmaradini\AppData\Local\Temp\ipykernel_2944\3495711940.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_sexo_mes_agrupado = df_sexo_mes.groupby(["mes_pesquisa", "sexo"])[sintomas].sum().reset_index()


In [37]:
from io import BytesIO
import boto3

# Cliente S3
s3_client = boto3.client("s3")
bucket_name = "techchallange-637423401077"  # só o bucket

# Dicionário com nomes e dataframes
dataframes = {
    "df_sexo_agrupado.parquet": df_sexo_agrupado,
    "df_mes_agrupado.parquet": df_mes_agrupado,
    "df_sexo_mes_agrupado.parquet": df_sexo_mes_agrupado
}

# Prefixo (pasta) dentro do bucket
prefix = "gold/"

# Loop para salvar todos no S3
for file_name, df in dataframes.items():
    buffer = BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)
    s3_client.put_object(Bucket=bucket_name, Key=prefix + file_name, Body=buffer.getvalue())
    print(f"DataFrame salvo em s3://{bucket_name}/{prefix}{file_name}")


DataFrame salvo em s3://techchallange-637423401077/gold/df_sexo_agrupado.parquet
DataFrame salvo em s3://techchallange-637423401077/gold/df_mes_agrupado.parquet
DataFrame salvo em s3://techchallange-637423401077/gold/df_sexo_mes_agrupado.parquet
